In [ ]:
{-# LANGUAGE BangPatterns, ScopedTypeVariables #-}
import Control.Arrow (Arrow(..))
import Control.Monad
import Control.Monad.Primitive
import Control.Parallel.Strategies
import Control.Concurrent
import Control.Exception (evaluate)

import Data.List
import Data.Ord
import qualified Data.Set as Set
import qualified Data.Vector.Unboxed as U

import Numeric.SpecFunctions
import Numeric.MathFunctions.Constants
import Numeric.MathFunctions.Comparison
import Numeric.Polynomial
import Numeric.RootFinding

import Text.Printf(printf)

import IHaskell.Display
import Graphics.Rendering.Chart.Backend.Cairo
import Graphics.Rendering.Chart.Easy hiding (within)

import Debug.Trace

:l NB/Plot
:l NB/Heatmap

greyColormap 0.2
_heat_map_values (def :: HeatMap Double Double Double)

setNumCapabilities 6

In [ ]:
-- | Data type to describe approximation used. See 
data Guess
  = AS109
  | AS64_2
  | NR1
  | NR2
  deriving (Show,Eq)

data Res = Res
  { result :: !Double
  , nIter  :: !Int
  , iters  :: [Double]
  }
  deriving Show 

-- | Compute inverse of regularized incomplete beta function. Uses
-- initial approximation from AS109, AS64 and Halley method to solve
-- equation.
invIncompleteBeta' 
  :: (Double -> Double -> Double -> (Guess,Double)) -- Initial guess
  -> Int
  -> Double     -- ^ /p/ > 0
  -> Double     -- ^ /q/ > 0
  -> Double     -- ^ /a/ ∈ [0,0.5] !!!
  -> Res
invIncompleteBeta' oracle nLim p q a
  | p <= 0 || q <= 0 =
      error $ printf "invIncompleteBeta p <= 0 || q <= 0.  p=%g q=%g a=%g" p q a
  | a <  0 || a >  1 =
      error $ printf "invIncompleteBeta x must be in [0,1].  p=%g q=%g a=%g" p q a
  | a == 0 || a == 1 = Res a 0 []
  | a > 0.5          = error "not implemented"
  | otherwise        = invIncompleteBetaWorker oracle nLim (logBeta p q) p q  a


invIncompleteBetaWorker
  :: (Double -> Double -> Double -> (Guess,Double))
  -> Int 
  -> Double -> Double -> Double -> Double -> Res
-- NOTE: p <= 0.5.
invIncompleteBetaWorker oracle nLim beta a b p = loop (0::Int) [] (snd $ oracle a b p)
  where
    a1 = a - 1
    b1 = b - 1
    -- Solve equation using Halley method
    done i xs x = Res { result = x 
                      , nIter  = i
                      , iters  = reverse $ x : xs
                      }
    loop !i !xs !x
      -- We cannot continue at this point so we simply return `x'
      -- | traceShow (x,f,f',u) False = undefined                  
      | x == 0 || x == 1             = done i xs x      
      -- When derivative becomes infinite we cannot continue
      -- iterations. It can only happen in vicinity of 0 or 1. It's
      -- hardly possible to get good answer in such circumstances but
      -- `x' is already reasonable.
      | isInfinite f'                = done i xs x
      -- Iterations limit reached. Most of the time solution will
      -- converge to answer because of discreteness of Double. But
      -- solution have good precision already.
      | i >= nLim                     = done i xs x
      -- Solution converges      
      | isNaN x' = error $ "Got to the NaN a="++show a++" b="++show b++" p="++show p++" x_last="++show x ++
         "  \nf="++show f ++ "  f'="++show f' ++ "u = " ++ show u
      | abs dx <= 16 * m_epsilon * x = done i xs x'
      | otherwise                    = loop (i+1) (x:xs) x'
      where
        -- Calculate Halley step.
        f   = incompleteBeta a b x  - p
        f'  = exp $ a1 * log x + b1 * log1p (-x) - beta
        u   = f / f'
        -- dx  = u / (1 - 0.5 * min 1 (u * (a1 / x - b1 / (1 - x))))
        corr | d > 1 = 1
             | d < -1 = -1
             | isNaN d = 0
             | otherwise = d
          where d = u * (a1 / x - b1 / (1 - x))
        dx  = u / (1 - 0.5 * corr)
        -- dx  = u / (1 - 0.5 * min 1 (u * (a1 / x - b1 / (1 - x))))
        -- Next approximation. If Halley step leads us out of [0,1]
        -- range we revert to bisection.
        x'  | z < 0     = x / 2
            | z > 1     = (x + 1) / 2
            | otherwise = z
            where z = x - dx

-- Calculate initial guess. Approximations from AS64, AS109 and
-- Numerical recipes are used.
--
-- Equations are referred to by name of paper and number e.g. [AS64 2]
-- In AS64 papers equations are not numbered so they are refered
-- to by number of appearance starting from definition of
-- incomplete beta.
guessIIBeta :: Double -> Double -> Double -> (Guess,Double)
guessIIBeta a b p
      -- In this region we use approximation from AS109 (Carter
      -- approximation). It's reasonably good (2 iterations on
      -- average)
      | a > 1 && b > 1 =
          let r = (y*y - 3) / 6
              s = 1 / (2*a - 1)
              t = 1 / (2*b - 1)
              h = 2 / (s + t)
              w = y * sqrt(h + r) / h - (t - s) * (r + 5/6 - 2 / (3 * h))
          in (AS109, a / (a + b * exp(2 * w)))
      -- Otherwise we revert to approximation from AS64 derived from
      -- [AS64 2] when it's applicable.
      --
      -- It slightly reduces average number of iterations when `a' and
      -- `b' have different magnitudes.
      | chi2 > 0 && ratio > 1 = (AS64_2, 1 - 2 / (ratio + 1))
      -- If all else fails we use approximation from "Numerical
      -- Recipes". It's very similar to approximations [AS64 4,5] but
      -- it never goes out of [0,1] interval.
      | otherwise = case () of
          _| p < t / w  -> (NR1, (a * p * w) ** (1/a))
           | otherwise  -> (NR2, 1 - (b * (1 - p) * w) ** (1/b))
           where
             lna = log $ a / (a+b)
             lnb = log $ b / (a+b)
             t   = exp( a * lna ) / a
             u   = exp( b * lnb ) / b
             w   = t + u
      where
        -- Formula [2]
        ratio = (4*a + 2*b - 2) / chi2
        -- Quantile of chi-squared distribution. Formula [3].
        chi2 = 2 * b * (1 - t + y * sqrt t) ** 3
          where
            t   = 1 / (9 * b)
        -- `y' is Hasting's approximation of p'th quantile of standard
        -- normal distribution.
        y   = r - ( 2.30753 + 0.27061 * r )
                  / ( 1.0 + ( 0.99229 + 0.04481 * r ) * r )
          where
            r = sqrt $ - 2 * log p

In [ ]:
plotRoundTripError nLim oracle a b = toRenderable $
  let fun x = let p  = result $ invIncompleteBeta' oracle nLim a b x
                  x' = incompleteBeta    a b p
              in x'
  in  plotFunctionsLog [\x -> logBase 10 $ relativeError (fun x) x] (1e-10, 0.5)


plotIterationsErr nLim oracle aRng bRng n
  = layout_title .~ -- ("a = " ++ show aRng ++ " b = " ++ show bRng ++ " max=" ++ show maxN)
      (printf "%.3g - %.3g" minN maxN)
  $ layout_plots .~ 
    [ toPlot $ heat_map_values    .~ nIterations
             $ heat_map_color_map .~ blackbodyColormap
             $ def
             ]
  $ def
  where
    maxN = maximum [z | (_,_,z) <- nIterations]
    minN = minimum [z | (_,_,z) <- nIterations]
    call a b x
      | incompleteBeta a b m_tiny > x/4 = []
      | otherwise                       = [relativeError x (incompleteBeta a b y)]
      where
        y = result $ invIncompleteBeta' oracle nLim a b x               
    nIterations = flip using (parList rdeepseq) 
      [ ((a1,b1),(a2,b2), maximum iters)
      | (a1,a,a2) <- linspaceIntervals aRng n
      , (b1,b,b2) <- linspaceIntervals bRng n
        -- Calculate number of iterations
      , let iters = concatMap (\x -> call a b x)
                  $ logspace (1e-10,0.5) 100
--      , sqrt (min a b) > abs (a - b)
      ]

---------------------------------------------------------------------
dropLoop xs
  = map fst
  $ takeWhile (\(x,vals) -> not $ Set.member x vals)
  $ zip xs sets
  where
    sets = scanl (flip Set.insert) Set.empty xs
    
plotNIterationsMap oracle aRng bRng n
  = layout_title .~ printf "MAX %.3g - %.3g" minN maxN
  $ layout_plots .~ 
    [ toPlot $ heat_map_values    .~ nIterations
             $ heat_map_color_map .~ blackbodyColormap
             $ def
             ]
  $ def
  where
    extract is | n > 50    = traceShow x n
               | otherwise = n
      where (x,n) = maximumBy (comparing snd) is
    maxN = maximum [z | (_,_,z) <- nIterations]
    minN = minimum [z | (_,_,z) <- nIterations]
    nIterations = flip using (parList rdeepseq) $ 
      [ ((a1,b1),(a2,b2), fromIntegral (extract iters) :: Double)
      | (a1,a,a2) <- linspaceIntervals aRng n
      , (b1,b,b2) <- linspaceIntervals bRng n
        -- Calculate number of iterations
      , let iters = map (\x -> ((x,a,b), cleverNIter $ invIncompleteBeta' oracle 100 a b x))
                  $ logspace (1e-10,0.5) 100
      --, sqrt (min a b) > abs (a - b)
      ]

cleverNIter :: Res -> Int
cleverNIter r 
      | n == 100  = length $ dropLoop $ iters r
      | otherwise = n
      where
        n   = nIter r
        
plotAvgIterationsMap oracle aRng bRng n
  = layout_title .~ printf "AVG %.3g - %.3g" minN maxN
  $ layout_plots .~ 
    [ toPlot $ heat_map_values    .~ nIterations
             $ heat_map_color_map .~ blackbodyColormap
             $ heat_map_zrange    .~ (Nothing,Just 7)
             $ def
             ]
  $ def
  where
    extract is = sum (map (fromIntegral . snd) is) / fromIntegral (length is)
    minN = minimum [z | (_,_,z) <- nIterations]
    maxN = maximum [z | (_,_,z) <- nIterations]
    nIterations = flip using (parList rdeepseq) $ 
      [ ((a1,b1),(a2,b2), extract iters :: Double)
      | (a1,a,a2) <- linspaceIntervals aRng n
      , (b1,b,b2) <- linspaceIntervals bRng n
        -- Calculate number of iterations
      , let iters = map (\x -> ((x,a,b), cleverNIter $ invIncompleteBeta' oracle 100 a b x))
                  $ logspace (1e-10,0.5) 100
      --, sqrt (min a b) > abs (a - b)
      ]

In [ ]:
-- Oracle for [0,1]×[0,1]
oracleBoost11 a b p 
  | x >= 0 && x <= 1 = (AS109, x)
  | otherwise = error $ printf "oracle 11': a=%g b=%b p=%g : x=%g" a b p x
  where
    xg    = (a * p * exp (logBeta a b)) ** (1/a)    
    guess = xg / (1 + xg)
    beta = exp $ logBeta a b
    -- Inflection point
    x_infl = (1 - a) / (2 - a - b)
    p_infl = incompleteBeta a b x_infl
    x | p < p_infl = let xg = (a * p * beta)  ** (1/a)    in xg / (1+xg)
      | otherwise  = let xg = (b * (1-p) * beta) ** (1/b) in 1 - xg/(1+xg)
      
      
oracleBoost11pp a b p
  | p < p_infl = x0
  | otherwise  = 1-x1
  where
    x_infl = (a - 1) / (a + b - 2)
    p_infl = incompleteBeta a b x_infl
    --
    x0 = exp $ (log(p * a) + logBeta a b) / a
    x1 = exp $ (log((1-p) * b) + logBeta a b) / b
    
oracleBoost11rest a b p
  | x == 0    = m_tiny
  | otherwise = x
  where
    beta = exp (logBeta a b)
        {-
    x | b < a = case () of
        _| p**(1/b) < 0.5 -> 1 - ((1-p) * b * beta) ** (1/b)
         | otherwise      -> (1 - (1-p) ** (a * beta))**(1/a)
      | otherwise = case () of
        _| p**(1/a) < 0.5 -> (p * a * beta) ** (1/a)
         | otherwise      -> 1 - (1 - p ** (b * beta))**(1/b)
    -}
    x | b < a = case () of
        _| p**b < 0.5   -> (p * a * beta) ** (1/a)
         | otherwise    -> (1 - (1-p) ** (a * beta))**(1/a) 
      | otherwise = case () of
        _| p**(1/a) < 0.5 -> (p * a * beta) ** (1/a)
         | otherwise      -> 1 - (1 - p ** (b * beta))**(1/b)         
         {-
toRenderable $
  let a    = 1.125
      b    = 0.125
      beta = exp (logBeta a b)
      func p | p**(1/a) < 0.5 = (p * a * beta) ** (1/a)
             | otherwise      = 1 - (1 - p ** (b * beta))**(1/b)
  in plotFunctions [ invIncompleteBeta a b
                   , func
             --      , oracleBoost11pp a b
                   ] (0,1)    
    x | p**(1/a) < 0.5 = (p * a * beta) ** (1/a)
      | otherwise      = 1 - (1 - p ** (b * beta))**(1/b)
      -}

In [ ]:

temmeSect4 aa bb pp
  | aa > bb   = x0
  | otherwise = 1 - x0
--  = traceShow (aa,bb,p,"U",u,eta0,eta) $ traceShow x0 x0
  where
    a = max aa bb
    b = min aa bb    
    p | aa > bb   = 1 - pp
      | otherwise = pp 
    -- 
    eta0 = invIncompleteGamma b p / a
    mu   = b / a
    w    = sqrt(1 + mu);
    w_2  = w * w
    w_3  = w_2 * w
    w_4  = w_2 * w_2
    w_5  = w_3 * w_2
    w_6  = w_3 * w_3
    w_7  = w_4 * w_3
    w_8  = w_4 * w_4
    w_9  = w_5 * w_4
    w_10 = w_5 * w_5
    d    = eta0 - mu
    d_2  = d * d
    d_3  = d_2 * d
    d_4  = d_2 * d_2
    w1   = w + 1
    w1_2 = w1 * w1
    w1_3 = w1 * w1_2
    w1_4 = w1_2 * w1_2
    --
    e1 = (w + 2) * (w - 1) / (3 * w)
       + (w_3 + 9 * w_2 + 21 * w + 5) * d / (36 * w_2 * w1)
       - (w_4 - 13 * w_3 + 69 * w_2 + 167 * w + 46) * d_2 / (1620 * w1_2 * w_3)
       - (7 * w_5 + 21 * w_4 + 70 * w_3 + 26 * w_2 - 93 * w - 31) * d_3 / (6480 * w1_3 * w_4)
       - (75 * w_6 + 202 * w_5 + 188 * w_4 - 888 * w_3 - 1345 * w_2 + 118 * w + 138) * d_4 / (272160 * w1_4 * w_5)
    e2 = (28 * w_4 + 131 * w_3 + 402 * w_2 + 581 * w + 208) * (w - 1) / (1620 * w1 * w_3)
       - (35 * w_6 - 154 * w_5 - 623 * w_4 - 1636 * w_3 - 3983 * w_2 - 3514 * w - 925) * d / (12960 * w1_2 * w_4)
       - (2132 * w_7 + 7915 * w_6 + 16821 * w_5 + 35066 * w_4 + 87490 * w_3 + 141183 * w_2 + 95993 * w + 21640) * d_2  / (816480 * w_5 * w1_3)
       - (11053 * w_8 + 53308 * w_7 + 117010 * w_6 + 163924 * w_5 + 116188 * w_4 - 258428 * w_3 - 677042 * w_2 - 481940 * w - 105497) * d_3 / (14696640 * w1_4 * w_6)
    e3 = -((3592 * w_7 + 8375 * w_6 - 1323 * w_5 - 29198 * w_4 - 89578 * w_3 - 154413 * w_2 - 116063 * w - 29632) * (w - 1)) / (816480 * w_5 * w1_2)
       - (442043 * w_9 + 2054169 * w_8 + 3803094 * w_7 + 3470754 * w_6 + 2141568 * w_5 - 2393568 * w_4 - 19904934 * w_3 - 34714674 * w_2 - 23128299 * w - 5253353) * d / (146966400 * w_6 * w1_3)
       - (116932 * w_10 + 819281 * w_9 + 2378172 * w_8 + 4341330 * w_7 + 6806004 * w_6 + 10622748 * w_5 + 18739500 * w_4 + 30651894 * w_3 + 30869976 * w_2 + 15431867 * w + 2919016) * d_2 / (146966400 * w1_4 * w_7)
    eta = evaluatePolynomialL (1/a) [eta0, e1, e2, e3]
    polyW coef = evaluatePolynomialL w    
    --
    u = eta - mu * log eta + (1 + mu) * log(1 + mu) - mu
    cross = 1 / (1 + mu);
    lower = if eta < mu then cross else 0
    upper = if eta < mu then 1     else cross
    x_guess = (lower + upper) / 2    
    func x = ( u + log x + mu*log(1 - x)
             , 1/x - mu/(1-x)
             )
    Root x0 = newtonRaphson 1e-8 (lower, x_guess, upper) func    

newOracle a b p
  | a+b>5 && a/b > 4    = (AS109, temmeSect4    a b p)  
-- It loses precision in somewhat awful manner
  | a+b>5 && a>1 && a/b < 0.25 = (AS109, temmeSect4    a b p)
  | a<1 && b < 1     = oracleBoost11 a b p
  | a+b < 6 && a>1 && b>1 = (AS109, oracleBoost11pp a b p)
  | a+b < 5 && b>a   = (AS109, oracleBoost11rest a b p)
  | otherwise        = guessIIBeta   a b p

In [ ]:
oracleNN a b p = (AS109, invIncBetaGuess (logBeta a b) a b p)

toRenderable $ plotIterationsErr 0   guessIIBeta   (0,10) (0,10) 40
toRenderable $ plotIterationsErr 0   oracleNN     (0,10) (0,10) 40
toRenderable $ plotNIterationsMap    guessIIBeta   (0,10) (0,10) 40
toRenderable $ plotNIterationsMap    oracleNN     (0,10) (0,10) 40
toRenderable $ plotAvgIterationsMap  guessIIBeta   (0,10) (0,10) 40
toRenderable $ plotAvgIterationsMap  oracleNN      (0,10) (0,10) 40

In [ ]:
toRenderable $
  let a = 15
      b = 1
  in plotFunctions [ temmeSect4 a b
                   , invIncompleteBeta a b
                   ] (0.0001,0.9999)
toRenderable $
  let a = 15
      b = 1
  in plotFunctions [ \x -> logBase 10 $ relativeError (temmeSect4 a b x) (invIncompleteBeta a b x)
                   ] (0.0001,0.9999)

In [ ]:
toRenderable $
  let a = 3
      b = 0.01
  in plotFunctions [ \x -> fromIntegral $ cleverNIter $ invIncompleteBeta' newOracle 100 a b x
                   ] (0.0001,0.5)

In [ ]:
toRenderable $
  let a = 0.1
      b = 15
  in plotFunctions [ temmeSect4 a b
                   , invIncompleteBeta a b
                   ] (0.0001,0.9999)
toRenderable $
  let a = 0.1
      b = 15
  in plotFunctions [ \x -> logBase 10 $ abs $ relativeError (invIncompleteBeta a b x) (temmeSect4 a b x)
                   ] (0.0001,0.9999)

In [ ]:
toRenderable $ plotIterationsErr 0   guessIIBeta   (0,20) (0.1,3) 30
toRenderable $ plotIterationsErr 10  guessIIBeta   (0,20) (0.1,3) 30
toRenderable $ plotIterationsErr 0   newOracle     (0,20) (0.1,3) 30
toRenderable $ plotIterationsErr 10  newOracle     (0,20) (0.1,3) 30

In [ ]:
toRenderable $ plotIterationsErr 0   newOracle     (5,10) (0.05,0.2) 30
toRenderable $ plotIterationsErr 10  newOracle     (5,10) (0.05,0.2) 30

In [ ]:
toRenderable $ plotIterationsErr 0   guessIIBeta   (0,10) (0,10) 40
toRenderable $ plotIterationsErr 0   newOracle     (0,10) (0,10) 40
toRenderable $ plotNIterationsMap    guessIIBeta   (0,10) (0,10) 40
toRenderable $ plotNIterationsMap    newOracle     (0,10) (0,10) 40
toRenderable $ plotAvgIterationsMap  guessIIBeta   (0,10) (0,10) 40
toRenderable $ plotAvgIterationsMap  newOracle     (0,10) (0,10) 40

In [ ]:
newOracle2 a b p
  | a+b>5 && a/b > 4    = (AS109, temmeSect4    a b p)  
-- It loses precision in somewhat awful manner
  | a+b>5 && a/b < 0.25 = (AS109, temmeSect4    a b p)
  | a<1 && b < 1     = oracleBoost11 a b p
  | a+b < 5 && a>1 && b>1 = (AS109, oracleBoost11pp a b p)
  | a+b < 5 && b>a   = (AS109, oracleBoost11rest a b p)
  | otherwise        = guessIIBeta   a b p
toRenderable $ plotIterationsErr 0   guessIIBeta    (0,2) (0,100) 40
toRenderable $ plotIterationsErr 0   newOracle2     (0,2) (0,100) 40

In [ ]:
toRenderable $ plotIterationsErr 10  oracleBoost11 (0,1) (0.1,1) 30
toRenderable $ plotIterationsErr 100 oracleBoost11 (0,1) (0.1,1) 30
toRenderable $ plotIterationsErr 10  guessIIBeta   (0,1) (0.1,1) 30

In [ ]:
plotRoundTripError 10 oracleBoost11 0.61 0.01

In [ ]:
toRenderable $ plotIterationsErr 0   guessIIBeta  (1,100) (1,100) 20
toRenderable $ plotIterationsErr 10  guessIIBeta  (1,100) (1,100) 20

In [ ]:
toRenderable $ plotIterationsErr 0  guessIIBeta (10,1000) (10,1000) 20
toRenderable $ plotIterationsErr 10 guessIIBeta (10,1000) (10,1000) 20

In [ ]:
toRenderable $ plotIterationsErr 0  guessIIBeta (10,10000) (10,10000) 30
toRenderable $ plotIterationsErr 10 guessIIBeta (10,10000) (10,10000) 30

In [ ]:
toRenderable $ plotIterationsErr 10  guessIIBeta (0.1,1) (0.1,1) 30
toRenderable $ plotIterationsErr 100 guessIIBeta (0.1,1) (0.1,1) 30

In [ ]:
toRenderable $ plotIterationsErr 10  guessIIBeta (0,20) (0.1,2) 30
toRenderable $ plotIterationsErr 100 guessIIBeta (0,20) (0.1,2) 30

In [ ]:
toRenderable $ plotIterationsErr 10  guessIIBeta (6,8) (0.05,0.1) 60
toRenderable $ plotIterationsErr 100 guessIIBeta (6,8) (0.05,0.1) 60

In [ ]:
toRenderable $ plotIterationsErr 0   guessIIBeta (0,50) (0.1,4) 30
toRenderable $ plotIterationsErr 10  guessIIBeta (0,50) (0.1,4) 30
toRenderable $ plotIterationsErr 100 guessIIBeta (0,50) (0.1,4) 30

In [ ]:
toRenderable $ plotIterationsErr 10  guessIIBeta (0.1,4) (0,40) 30
toRenderable $ plotIterationsErr 100 guessIIBeta (0.1,4) (0,40) 30

In [ ]:
toRenderable $ plotIterationsErr 10  guessIIBeta (0,5) (0.1,4) 30
toRenderable $ plotIterationsErr 100 guessIIBeta (0,5) (0.1,4) 30

In [ ]:
toRenderable $ plotIterationsErr 0  guessIIBeta (1,20) (1,20) 60

In [ ]:
oracleBoost11' a b p 
  | x >= 0 && x <= 1 = (AS109, x)
  | otherwise = error $ printf "oracle 11': a=%g b=%b p=%g : x=%g" a b p x
  where
    xg    = (a * p * exp (logBeta a b)) ** (1/a)    
    guess = xg / (1 + xg)
    beta = exp $ logBeta a b
    -- Inflection point
    x_infl = (1 - a) / (2 - a - b)
    p_infl = incompleteBeta a b x_infl
    x | p < p_infl = let xg = (a * p * beta)  ** (1/a)    in xg / (1+xg)
      | otherwise  = let xg = (b * (1-p) * beta) ** (1/b) in 1 - xg/(1+xg)

In [ ]:
toRenderable $ plotIterationsErr 0 guessIIBeta    (0,1) (0,1) 40
toRenderable $ plotIterationsErr 0 oracleBoost11  (0,1) (0,1) 40
toRenderable $ plotIterationsErr 0 oracleBoost11' (0,1) (0,1) 40

In [ ]:
toRenderable $ plotNIterationsMap guessIIBeta    (0.05,1) (0,1) 40
toRenderable $ plotNIterationsMap oracleBoost11' (0.05,1) (0,1) 40

In [ ]:
toRenderable $ plotAvgIterationsMap guessIIBeta    (0.05,1) (0,1) 40
toRenderable $ plotAvgIterationsMap oracleBoost11' (0.05,1) (0,1) 40

In [ ]:
toRenderable $
  let a = 0.3625
      b = 0.95
      beta = exp $ logBeta a b
      --
      x_infl = (1 - a) / (2 - a - b)
      p_infl = incompleteBeta a b x_infl
      appr p | p < p_infl = let xg = (a * p * beta)  ** (1/a)    in xg / (1+xg)
             | otherwise  = let xg = (b * (1-p) * beta) ** (1/b) in 1 - xg/(1+xg)
      appr1 p = let xg = (a * p * beta)  ** (1/a)    in xg / (1+xg)
      appr2 p = let xg = (b * (1-p) * beta) ** (1/b) in 1 - xg/(1+xg)
  in plotFunctions [ invIncompleteBeta a b
                   , appr
                   , appr1
                   , appr2
                   ] (0,1)

In [ ]:
sqrt m_epsilon

In [ ]:
oracleBoost11pp a b p
  | p < p_infl = x0
  | otherwise  = 1-x1
  where
    x_infl = (a - 1) / (a + b - 2)
    p_infl = incompleteBeta a b x_infl
    --
    x0 = exp $ (log(p * a) + logBeta a b) / a
    x1 = exp $ (log((1-p) * b) + logBeta a b) / b
toRenderable $
  let a = 1.7
      b = 3
  in plotFunctions [ invIncompleteBeta a b
                   , oracleBoost11pp a b
                   ] (0,1)
{-
  T ap1 = a - 1;
         T bm1 = b - 1;
         T a_2 = a * a;
         T a_3 = a * a_2;
         T b_2 = b * b;
         T terms[5] = { 0, 1 };
         terms[2] = bm1 / ap1;
         ap1 *= ap1;
         terms[3] = bm1 * (3 * a * b + 5 * b + a_2 - a - 4) / (2 * (a + 2) * ap1);
         ap1 *= (a + 1);
         terms[4] = bm1 * (33 * a * b_2 + 31 * b_2 + 8 * a_2 * b_2 - 30 * a * b - 47 * b + 11 * a_2 * b + 6 * a_3 * b + 18 + 4 * a - a_3 + a_2 * a_2 - 10 * a_2)
                    / (3 * (a + 3) * (a + 2) * ap1);
         x = tools::evaluate_polynomial(terms, x, 5);
-}


In [ ]:
toRenderable $
  let a = 0.7
      b = 3
      beta = exp (logBeta a b)
      func p | p**(1/a) < 0.5 = (p * a * beta) ** (1/a)
             | otherwise      = 1 - (1 - p ** (b * beta))**(1/b)
  in plotFunctions [ invIncompleteBeta a b
                   , func
             --      , oracleBoost11pp a b
                   ] (0,1)

In [ ]:
toRenderable $
  let a    = 1.125
      b    = 0.125
      beta = exp (logBeta a b)
      func p | p**(1/b) < 0.5 = 1 - ((1-p) * b * beta) ** (1/b)
             | otherwise      = (1 - (1-p) ** (a * beta))**(1/a)
  in plotFunctions [ invIncompleteBeta a b
                   , func
                   , oracleBoost11rest a b
                   ] (0,1)

In [ ]:
toRenderable $
  let a    = 0.125
      b    = 3.125
      beta = exp (logBeta a b)
      func p | p**(1/a) < 0.5 = (p * a * beta) ** (1/a)
             | otherwise      = 1 - (1 - p ** (b * beta))**(1/b)
  in plotFunctions [ invIncompleteBeta a b
                   , func
                   , oracleBoost11rest a b
                   ] (0,1)

In [ ]:
toRenderable $
  let a    = 6
      b    = 1
      beta = exp (logBeta a b)
      func1 p = (p * a * beta) ** (1/a)
      func2 p = 1 - (1 - p ** (b * beta))**(1/b)
      func p | p**(1/b) > 0.5 = (p * a * beta) ** (1/a)
             | otherwise      = 1 - (1 - p ** (b * beta))**(1/b)
  in plotFunctions [ invIncompleteBeta a b
                   , func2
                   -- , func2
                   , oracleBoost11rest a b
                   ] (0,1)

In [ ]:
oracleBoost11rest 4 0.125 0

In [ ]:
test a b p
  | x == 0    = m_tiny
  | otherwise = x
  where
    beta = exp (logBeta a b)
    x | b < a = case () of
        _| p**b < 0.5   -> (p * a * beta) ** (1/a)
         | otherwise    -> (1 - (1-p) ** (a * beta))**(1/a) 
      | otherwise = case () of
        _| p**(1/a) < 0.5 -> (p * a * beta) ** (1/a)
         | otherwise      -> 1 - (1 - p ** (b * beta))**(1/b)
toRenderable $
  let a    = 4
      b    = 0.9
  in plotFunctions [ invIncompleteBeta a b
                   , test a b
                   ] (0,1)